# Extraction + Learning : Hardware Spec Sheets

This notebook demonstrates the full entity extraction process on transistor data sheets, extracting min storage temperatures.

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# from snorkel import SnorkelSession
# session = SnorkelSession()

### Extraction

In [1]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/hardware/hardware_docs/')

In [2]:
from snorkel.parser import TableParser
table_parser = TableParser()

In [3]:
from snorkel.parser import CorpusParser
cp = CorpusParser(html_parser, table_parser, max_docs=10)
%time corpus = cp.parse_corpus(name='Hardware Corpus')

CPU times: user 4.54 s, sys: 96 ms, total: 4.64 s
Wall time: 7.27 s


In [4]:
# session.add(corpus)
# session.commit()

In [5]:
# from snorkel.models import Corpus
# corpus = session.query(Corpus).filter(Corpus.name == 'CDR Corpus').one()
# corpus

In [6]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import NumberMatcher, RangeMatcher

# Define a candidate space
ngrams = TableNgrams(n_max=3)

# Define a matcher
number_matcher = RangeMatcher(low=-100,high=-10)

In [7]:
# from snorkel.candidates import Candidates
# %time candidates = Candidates(ngrams, number_matcher, corpus.get_contexts())
# for c in candidates.get_candidates()[:5]: print c
    
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(ngrams, number_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')
for cand in candidates[:10]: 
    print cand
print "%s candidates extracted" % len(candidates)

CPU times: user 1.62 s, sys: 121 ms, total: 1.74 s
Wall time: 1.68 s
Ngram("-40", context=None, chars=[77,79], words=[0,0])
Ngram("-40", context=None, chars=[123,125], words=[0,0])
Ngram("-55", context=None, chars=[297,299], words=[0,0])
Ngram("-40", context=None, chars=[160,162], words=[0,0])
Ngram("-10", context=None, chars=[228,230], words=[2,2])
Ngram("-40", context=None, chars=[249,251], words=[0,0])
Ngram("-10", context=None, chars=[315,317], words=[2,2])
Ngram("-30", context=None, chars=[389,391], words=[2,2])
Ngram("-50", context=None, chars=[414,416], words=[0,0])
Ngram("-30", context=None, chars=[469,471], words=[2,2])
40 candidates extracted


### Learning

First, generate gold data.

In [ ]:
from utils import collect_hardware_annotations
gt_dict = collect_hardware_annotations('data/hardware/gold_all.csv', 'stg_temp_min', candidates)

In [9]:
# read from csv

print "%s out of %s candidates have gold labels" % (len(gt_dict),len(candidates))
print "%s out of %s labeled candidates have positive label" \
        % (gt_dict.values().count(1), len(gt_dict))

98 gold annotations
1 out of 40 candidates have gold labels
0 out of 1 labeled candidates have positive label


Then confirm that features work:

In [17]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)
for f in featurizer.get_features_by_candidate(candidates[0])[:10]: print f

DDLIB_WORD_SEQ_[-40]
DDLIB_LEMMA_SEQ_[-40]
DDLIB_POS_SEQ_[CD]
DDLIB_DEP_SEQ_[ROOT]
DDLIB_W_LEFT_1_[_NUMBER]
DDLIB_W_LEFT_POS_1_[CD]
DDLIB_LENGTH_1
TABLE_ROW_NUM_1
TABLE_COL_NUM_2
TABLE_HTML_TAG_td


We now build a legacy DDLiteModel with which we will do learning.

In [ ]:
from snorkel.snorkel import DDLiteModel
DDL = DDLiteModel(candidates, featurizer.fit_transform(candidates), gt_dict)

DDL.set_holdout(idxs=range(50), validation_frac=0.5)
print "%s training data" % len(DDL.gt.get_labeled_ground_truth('training')[0])
print "%s test data" % len(DDL.gt.get_labeled_ground_truth('test')[0])
print "%s validation data" % len(DDL.gt.get_labeled_ground_truth('validation')[0])

Define labeling functions:

In [ ]:
def LF_to(m):
    return 1 if 'to' in m.post_window('words') else 0
def LF_storage(m):
    return 1 if 'storage' in m.aligned_ngrams('words') else -1
def LF_tstg(m):
    return 1 if 'tstg' in m.aligned_ngrams('words') else -1
def LF_temperature(m):
    return 1 if 'temperature' in m.aligned_ngrams('words') else -1

In [ ]:
LFs = [LF_to, LF_storage, LF_tstg, LF_temperature]
DDL.apply_lfs(LFs, clear=True)

In [ ]:
print DDL.lf_names
DDL.print_lf_stats()

In [ ]:
DDL.top_conflict_lfs(n=3)

In [ ]:
DDL.lowest_coverage_lfs(n=3)

Now learn, baby, learn!

In [ ]:
import matplotlib
import numpy as np
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,4)
mu_seq = np.ravel([1e-9, 1e-5, 1e-3, 1e-1])
lf_opts = {'sample': False, 'verbose': True}
model_opts = {'sample': False, 'n_iter': 3000, 'alpha': 0.5, 'mu': mu_seq, 'bias': False, 'verbose': True}
%time DDL.train_model(method='lr', lf_opts=lf_opts, model_opts=model_opts)

In [ ]:
DDL.show_log()
idxs, gt = DDL.get_labeled_ground_truth(subset=DDL.holdout())
acc_feats = np.mean(DDL.get_predicted(subset=DDL.holdout()) == gt)
print "Full model accuracy: {:.3f}".format(acc_feats)

Tune in next time for relation extraction!